# main implementation

In [6]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:10
🔁 Restarting kernel...


In [ ]:
!conda create -n my_env -c conda-forge python=3.11 numba cudatoolkit=11.8 -y

In [7]:
import os
import sys
import numba
import numpy as np
import math
import timeit

print("--- Environment Verification ---")
print(f"Python version: {sys.version}")
print(f"Numba version: {numba.__version__}")
# This command now runs INSIDE the clean environment and should find the correct compiler
os.system('which ptxas')
os.system('ptxas --version')
print("------------------------------\n")

from numba import jit, cuda
GPU_AVAILABLE = cuda.is_available()

--- Environment Verification ---
Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Numba version: 0.59.1
------------------------------



In [12]:
%%writefile main.py
import os
import sys
import numba
import numpy as np
import math
import timeit

print("--- Environment Verification ---")
print(f"Python version: {sys.version}")
print(f"Numba version: {numba.__version__}")
# This command now runs INSIDE the clean environment and should find the correct compiler
os.system('which ptxas')
os.system('ptxas --version')
print("------------------------------\n")

from numba import jit, cuda
GPU_AVAILABLE = cuda.is_available()

@jit(nopython=True, fastmath=True)
def s4_cpu_kernel(x: np.ndarray, derivative: bool = False, k: float = 5.0) -> np.ndarray:
    out = np.empty_like(x)
    for i in range(x.size):
        x_val = x[i]
        exp_neg_kx = math.exp(-k * x_val)
        exp_neg_x = math.exp(-x_val)
        a = 1.0 / (1.0 + exp_neg_kx)
        if not derivative:
            abs_x = math.fabs(x_val)
            one_plus_abs_x = 1.0 + abs_x
            softsign = x_val / one_plus_abs_x
            sigmoid = 1.0 / (1.0 + exp_neg_x)
            out[i] = a * softsign + (1.0 - a) * sigmoid
        else:
            one_minus_a = 1.0 - a
            da_dx = k * a * one_minus_a
            abs_x = math.fabs(x_val)
            one_plus_abs_x = 1.0 + abs_x
            softsign = x_val / one_plus_abs_x
            d_softsign = 1.0 / (one_plus_abs_x * one_plus_abs_x)
            sigmoid = 1.0 / (1.0 + exp_neg_x)
            d_sigmoid = sigmoid * (1.0 - sigmoid)
            out[i] = (da_dx * (softsign - sigmoid) + a * d_softsign + one_minus_a * d_sigmoid)
    return out

@cuda.jit(device=True)
def s4_cuda_device_function(x_val, derivative, k):
    exp_neg_kx = math.exp(-k * x_val)
    exp_neg_x = math.exp(-x_val)
    a = 1.0 / (1.0 + exp_neg_kx)
    if not derivative:
        abs_x = math.fabs(x_val)
        one_plus_abs_x = 1.0 + abs_x
        softsign = x_val / one_plus_abs_x
        sigmoid = 1.0 / (1.0 + exp_neg_x)
        return a * softsign + (1.0 - a) * sigmoid
    else:
        one_minus_a = 1.0 - a
        da_dx = k * a * one_minus_a
        abs_x = math.fabs(x_val)
        one_plus_abs_x = 1.0 + abs_x
        softsign = x_val / one_plus_abs_x
        d_softsign = 1.0 / (one_plus_abs_x * one_plus_abs_x)
        sigmoid = 1.0 / (1.0 + exp_neg_x)
        d_sigmoid = sigmoid * (1.0 - sigmoid)
        return (da_dx * (softsign - sigmoid) + a * d_softsign + one_minus_a * d_sigmoid)

@cuda.jit
def s4_cuda_kernel(x_array, out_array, derivative, k):
    idx = cuda.grid(1)
    if idx < x_array.size:
        out_array[idx] = s4_cuda_device_function(x_array[idx], derivative, k)

def s4_accelerated(x: np.ndarray, derivative: bool = False, k: float = 5.0) -> np.ndarray:
    if not isinstance(x, np.ndarray): x = np.array(x)
    if GPU_AVAILABLE:
        d_x = cuda.to_device(x)
        d_out = cuda.device_array_like(d_x)
        threads_per_block = 256
        blocks_per_grid = (x.size + (threads_per_block - 1)) // threads_per_block
        s4_cuda_kernel[blocks_per_grid, threads_per_block](d_x, d_out, derivative, k)
        return d_out.copy_to_host()
    else:
        return s4_cpu_kernel(x, derivative, k)

def s4_numpy(x: np.ndarray, derivative: bool = False, k: float = 5.0) -> np.ndarray:
    exp_neg_kx = np.exp(-k * x)
    exp_neg_x = np.exp(-x)
    a = 1 / (1 + exp_neg_kx)
    if not derivative:
        abs_x = np.abs(x)
        one_plus_abs_x = 1 + abs_x
        softsign = x / one_plus_abs_x
        sigmoid = 1 / (1 + exp_neg_x)
        return a * softsign + (1 - a) * sigmoid
    else:
        one_minus_a = 1 - a
        da_dx = k * a * one_minus_a
        abs_x = np.abs(x)
        one_plus_abs_x = 1 + abs_x
        softsign = x / one_plus_abs_x
        d_softsign = 1 / (one_plus_abs_x * one_plus_abs_x)
        sigmoid = 1 / (1 + exp_neg_x)
        d_sigmoid = sigmoid * (1.0 - sigmoid)
        return (da_dx * (softsign - sigmoid) + a * d_softsign + one_minus_a * d_sigmoid)


if __name__ == '__main__':
    array_size = 10_000_000
    test_data = np.linspace(-10, 10, array_size, dtype=np.float64)

    if not GPU_AVAILABLE:
        print("FATAL: GPU not available inside the Conda environment.")
    else:
        print("CUDA GPU Available: True")
        exec_path = "GPU"
        print(f"Execution path for accelerated function: {exec_path}\n")

        print("--- Correctness Check ---")
        numpy_result = s4_numpy(test_data)
        accelerated_result = s4_accelerated(test_data)

        if np.allclose(numpy_result, accelerated_result):
            print("✅ Correctness check PASSED. Outputs are numerically close.")
        else:
            print("❌ Correctness check FAILED.")
        print("-" * 25, "\n")

        print("--- Performance Benchmark ---")
        number_of_runs = 100
        accelerated_time = timeit.timeit('s4_accelerated(test_data)', globals=globals(), number=number_of_runs)
        numpy_time = timeit.timeit('s4_numpy(test_data)', globals=globals(), number=number_of_runs)
        print(f"Original NumPy version took: {numpy_time:.6f} seconds")
        print(f"Accelerated version ({exec_path}) took: {accelerated_time:.6f} seconds")
        if accelerated_time > 0:
          speedup = numpy_time / accelerated_time
          print(f"\nSpeedup: {speedup:.2f}x")

Overwriting main.py


In [13]:
!conda run -n my_env python main.py

/usr/local/cuda/bin/ptxas
ptxas: NVIDIA (R) Ptx optimizing assembler
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:14:54_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
--- Environment Verification ---
Python version: 3.11.14 | packaged by conda-forge | (main, Oct 22 2025, 22:46:25) [GCC 14.3.0]
Numba version: 0.62.1
------------------------------

CUDA GPU Available: True
Execution path for accelerated function: GPU

--- Correctness Check ---
✅ Correctness check PASSED. Outputs are numerically close.
------------------------- 

--- Performance Benchmark ---
Original NumPy version took: 37.019866 seconds
Accelerated version (GPU) took: 6.001073 seconds

Speedup: 6.17x



# with separated file (library)

In [21]:
%%writefile main.py
import numpy as np
import timeit
import sys

# --- Import from the local s4.py file ---
try:
    from s4 import S4Activation, s4, s4_numpy
except ImportError:
    print("FATAL ERROR: Could not import from s4.py.")
    print("Please make sure 's4.py' is in the same directory as 'main.py'.")
    sys.exit(1)

def run_demo_and_benchmark():
    """Demonstrates the usage of the S4Activation module and runs a benchmark."""
    print("--- S4 Activation Function Demo & Benchmark ---")

    # --- 1. Demonstration of the S4Activation Class ---
    print("\n--- 1. Class-based API Demo ---")

    s4_auto = S4Activation(device='auto')

    print(f"Is GPU available according to the activator? {s4_auto.is_gpu_available}")

    small_array = np.array([-2.0, -1.0, 0.0, 1.0, 2.0], dtype=np.float64)
    print(f"\nInput: {small_array}")
    print(f"s4(x): {s4_auto(small_array)}")
    print(f"s4'(x): {s4_auto(small_array, derivative=True)}")

    large_array = np.ones(s4_auto.gpu_threshold + 1)
    print("\nBackend selection logic demo:")
    print(f"Info for small array (size={small_array.size}): {s4_auto.get_backend_info(small_array)}")
    print(f"Info for large array (size={large_array.size}): {s4_auto.get_backend_info(large_array)}")

    # --- 2. Demonstration of the s4() convenience function ---
    print("\n--- 2. Functional API Demo ---")
    y_cpu = s4(small_array, device='cpu')
    print(f"s4(x) with device='cpu' forced: {y_cpu}")

    # --- 3. Performance Benchmark ---
    print("\n--- 3. Performance Benchmark (on a very large array) ---")

    benchmark_data = np.linspace(-10, 10, 10_000_000, dtype=np.float64)

    backend_info = s4_auto.get_backend_info(benchmark_data)
    print(f"Benchmarking with backend: '{backend_info['backend']}'")

    print("Performing a warm-up run (this includes compilation time)...")
    _ = s4_auto(benchmark_data)
    print("Warm-up complete.")

    number_of_runs = 100

    # --- CORRECTED PART ---
    # Create a dictionary of the local variables that timeit needs to see.
    timeit_namespace = {
        "s4_auto": s4_auto,
        "benchmark_data": benchmark_data,
        "s4_numpy": s4_numpy  # Also include the numpy function for its benchmark
    }

    # Time the accelerated version using the custom namespace
    accelerated_time = timeit.timeit(
        's4_auto(benchmark_data)',
        globals=timeit_namespace, # Pass the dictionary with our local variables
        number=number_of_runs
    )

    # Time the pure NumPy version using the same namespace
    numpy_time = timeit.timeit(
        's4_numpy(benchmark_data)',
        globals=timeit_namespace, # Pass the same dictionary
        number=number_of_runs
    )

    print(f"\nBaseline (NumPy): {numpy_time:.4f} seconds for {number_of_runs} runs.")
    print(f"Accelerated Path: {accelerated_time:.4f} seconds for {number_of_runs} runs.")

    if accelerated_time > 0 and 'numba' in sys.modules:
        speedup = numpy_time / accelerated_time
        print(f"\nSpeedup: {speedup:.2f}x")

    print("\n--- Benchmark Complete ---")

if __name__ == '__main__':
    run_demo_and_benchmark()

Overwriting main.py


In [22]:
!conda run -n my_env python main.py

--- S4 Activation Function Demo & Benchmark ---

--- 1. Class-based API Demo ---
Is GPU available according to the activator? True

Input: [-2. -1.  0.  1.  2.]
s4(x): [0.11916725 0.26379501 0.25       0.50154644 0.66667639]
s4'(x): [0.10481549 0.17140942 0.         0.24196223 0.11106223]

Backend selection logic demo:
Info for small array (size=5): {'backend': 'cpu', 'gpu_available': True, 'array_size': 5, 'array_shape': (5,), 'gpu_threshold': 10000, 'device_setting': 'auto'}
Info for large array (size=10001): {'backend': 'gpu', 'gpu_available': True, 'array_size': 10001, 'array_shape': (10001,), 'gpu_threshold': 10000, 'device_setting': 'auto'}

--- 2. Functional API Demo ---
s4(x) with device='cpu' forced: [0.11916725 0.26379501 0.25       0.50154644 0.66667639]

--- 3. Performance Benchmark (on a very large array) ---
Benchmarking with backend: 'gpu'
Performing a warm-up run (this includes compilation time)...
Warm-up complete.

Baseline (NumPy): 37.4321 seconds for 100 runs.
Accel